In [70]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [71]:
import Classification_Utils as cu
import MaxQuant_Postprocessing_Functions as mq
import numpy as np
from os import listdir
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.externals import joblib
from sklearn import preprocessing
import time

## Load and clean Data

In [72]:
HIGH_QUAL_DIR = 'F:\High_Quality\\' 
TEST_SET_DIR = 'F:\Cell_Line\\'

files_dir = HIGH_QUAL_DIR 
file_paths = listdir(files_dir) 

df = cu.combine_csvs(files_dir, file_paths)

df.dropna(axis='index', how='all', inplace=True) # drop any rows where all values are missing
df = df.drop(['\n'])

original_df = df.copy()

In [73]:
mq.log2_normalize(df)

df_min = df.min().min()
impute_val = df_min/2
df = df.fillna(impute_val)

# median normalize
mq.median_normalize(df)

C:\Users\kush494\Documents\Proteomics_Data_Processing\MaxQuant_Postprocessing_Functions.py:150: RuntimeWarning: divide by zero encountered in log2
  df.iloc[:,:] = np.log2(df.iloc[:,:])


In [74]:
tissues = ['Blood_Plasma', 'Blood_Serum', 'CSF', 'Liver', 'Monocyte', 'Ovary', 'Pancreas', 'Substantia_Nigra', 'Temporal_Lobe']
 
tissues_to_columns = cu.map_tissues_to_columns(df, tissues)

column_names = df.columns.values.tolist()
labels = cu.get_labels(column_names, tissues_to_columns)

In [1]:
NUM_SPLITS = 100

test_paths = listdir(TEST_SET_DIR) 
test_data = cu.combine_csvs(TEST_SET_DIR, test_paths)

test_labels = []
for col in test_data.columns.values.tolist():
    for tissue in tissues:
        if col.startswith(tissue):
            test_labels.append(tissue)
            continue

NameError: name 'listdir' is not defined

In [76]:
df.columns.values.tolist()

['Blood_Plasma_CPTAC_TrypDige_undepleted_normal_19Apr13_Methow_13-02-13',
 'Blood_Plasma_Darpa_2_human_02_23Jan17_Arwem_16-10-25',
 'Blood_Plasma_OMICS_EBV_HP_UW001_8Apr16_Arwen_16-01-03',
 'Blood_Plasma_OMICS_EBV_HP_UW002_8Apr16_Arwen_16-01-03',
 'Blood_Plasma_OMICS_EBV_HP_UW003_8Apr16_Arwen_16-01-03',
 'Blood_Plasma_OMICS_EBV_HP_UW004_8Apr16_Arwen_16-01-03',
 'Blood_Plasma_OMICS_EBV_HP_UW005_8Apr16_Arwen_16-01-03',
 'Blood_Plasma_OMICS_EBV_HP_UW006_8Apr16_Arwen_16-01-03',
 'Blood_Plasma_OMICS_EBV_HP_UW007_8Apr16_Arwen_16-01-03',
 'Blood_Plasma_OMICS_EBV_HP_UW008_8Apr16_Arwen_16-01-03',
 'Blood_Plasma_OMICS_EBV_HP_UW009_8Apr16_Arwen_16-01-03',
 'Blood_Plasma_OMICS_EBV_HP_UW010_8Apr16_Arwen_16-01-03',
 'Blood_Plasma_OMICS_EBV_HP_UW011_8Apr16_Arwen_16-01-03',
 'Blood_Plasma_OMICS_EBV_HP_UW012_8Apr16_Arwen_16-01-03',
 'Blood_Plasma_OMICS_EBV_HP_UW013_19Apr16_Arwen_16-01-03',
 'Blood_Plasma_OMICS_EBV_HP_UW014_8Apr16_Arwen_16-01-03',
 'Blood_Plasma_OMICS_EBV_HP_UW015_8Apr16_Arwen_16-01-03'

## Load classifiers

In [ ]:


model_path = finalized_model_folder + 'highqual_crossval_svc_grid.sav'

svc = joblib.load(open(model_path + '', 'svc'))



## Test classifier accuracy on iPS data

## PCA with Cell Line included

In [ ]:
### Copy test df, rename to cell_line
cell_line_df_copy = test_data.copy()
cell_line_df_copy.columns = rename_columns(cell_line_df_copy, 'Liver', 'Liver_Cell_Line')

### join dfs
combined_df = original_df.join(cell_line_df_copy, how='outer')

In [ ]:
### 2D and 3D PCA
scaled_data = combined_df.T

pca = PCA() # create a PCA object
pca.fit(scaled_data) # do the math
pca_data = pca.transform(scaled_data) # get PCA coordinates for dataframe

pca_3 = PCA(n_components=3) 
pca_3.fit(scaled_data)
pca_data_3 = pca_3.transform(scaled_data)

per_var, pca_labels = mq.make_scree_plot(pca, image_dir, 'Scree Plot - 25 Percentile')
per_var_3, pca_labels_3 = mq.make_scree_plot(pca_3, image_dir, '3 Dimensional Scree - 25 Percentile')

mq.draw_pca_graph2(column_names, pca_data, image_dir, column_to_color, per_var, pca_labels, tissues, tissues_to_columns, 'PCA - 25 Percentile')
#mq.draw_pca_graph(column_names, pca_data, image_dir, column_to_color, per_var, pca_labels, 'PCA_RobustScaler_Annotated')
mq.draw_3d_pca(column_names, pca_data_3, image_dir, column_to_color, per_var_3, pca_labels_3, tissues, tissues_to_columns, '3D PCA - 25 Percentile')